# Appendix B: Problem 14

Estimation of oil and gas viscosities at 2,500 psia and 190 deg F from scenario described by problems 11-13

In [6]:
import math
R = 10.73146 # Universal gas constant, psia-ft3/R-lbm mol

#Assumed
cgr = 40 # stb/mmscf
gamma_og = 0.739
gamma_gg = 0.80
cond_api = 60

# Given / Assumed
sg_gas_total = 0.85
p = 2500 #psia
t = 190 # deg F
api_oil = 36 # deg API

In [7]:
# Standing corrleation for wet gas
Tpchc = 187 + 330 * sg_gas_total - 71.5 * sg_gas_total**2
Ppchc = 706 - 51.7*sg_gas_total - 11.1*sg_gas_total**2

# And reduced properties are then
Tpr = (t+460)/Tpchc
Ppr = p/Ppchc

# Gas MW
Mg = 28.97 * sg_gas_total

In [8]:
# Grabbing the same code we previously wrote for Lucas correlation in Problem 7
squiggle = 9490*(Tpchc/(Mg**3*Ppchc**4))**(1/6)
ugsc_squiggle = (0.807*Tpr**0.618-0.357*math.exp(-0.449*Tpr)+0.340*math.exp(-4.058*Tpr)+0.018)
ugsc = ugsc_squiggle / squiggle
A1 = 1.245e-3*math.exp(5.1726*Tpr**-0.3286)/Tpr
A2 = A1 * (1.6553*Tpr - 1.2723)
A3 = 0.4489*math.exp(3.0578*Tpr**-37.7332)/Tpr
A4 = 1.7368*math.exp(2.231*Tpr**-7.6351)/Tpr
A5 = 0.9425*math.exp(-0.1853*Tpr**0.4489)
ug_on_ugsc = 1 + A1*Ppr**1.3088 / (A2*Ppr**A5 + (1 + A3*Ppr**A4)**-1)
ug_lucas = ug_on_ugsc * ugsc
print('Lucas Correlation Gas Vicosity:',round(ug_lucas,4), 'cP' )

Lucas Correlation Gas Vicosity: 0.021 cP


In [9]:
# Copying Hall-Yarborough Z factor solution from Problem 6
from scipy.optimize import fsolve
import math
def fy(y, *args):
    alpha, Pr, t = args
    return -alpha * Pr + (y + y ** 2 + y ** 3 - y ** 4) / (1 - y) ** 3 - (14.76 * t - 9.76 * t ** 2 + 4.58 * t ** 3) * y ** 2 + (90.7 * t - 242.2 * t ** 2 + 42.4 * t ** 3) * y ** (2.18 + 2.82 * t)
    
def Zfac(Tr, Pr):
    t = 1 / Tr
    alpha = 0.06125 * t * math.exp(-1.2 * (1 - t) ** 2)
    args = (alpha, Pr, t)
    y = 0.001
    y = fsolve(fy, y, args=args)
    return alpha * Pr / y

In [10]:
# And using Lee Gonzalez correlation (Eq 3.65)
A1 = (9.39 + 0.01607*Mg)*(t+460)**1.5/(209.2 + 19.26 *Mg + (t+460))
A2 = 3.448 + (986.4/(t+460)) + 0.01009*Mg
A3 = 2.447 - 0.2224*A2
Z = Zfac(Tpr, Ppr)[0]
rhog = p*Mg/(Z*R*(t+460))
ug_lee = A1 *1e-4 * math.exp(A2*(rhog/62.4)**A3) # Eq 3.65a where rhog is in SG relative to water
print('Lee-Gonzalez Correlation Gas Vicosity:',round(ug_lee,4), 'cP' )

Lee-Gonzalez Correlation Gas Vicosity: 0.0212 cP


In [11]:
# Now, oil viscosity. First the dead oil viscosity with Bergman Eq 3.119
A0 = 22.33 - 0.194*api_oil + 0.00033*api_oil**2
A1 = -3.20 + 0.0185 * api_oil
uod = math.exp(math.exp(A0 + A1 * math.log(t + 310))) - 1 # Rearrangement of Eq 3.119, yes it is a double logarithm, not a misprint

# Now, estimate solution gor with Standing correlation, assuming oil at bubble point
Rs = sg_gas_total*(((0.055*p + 1.4)*10**(0.0125*api_oil))/10**(0.00091*t))**1.205 # ** Note: Example calculation missing the 1.205 exponent

# And the live oil @ Pb using Chew & Connally correlation with Bergman equation constants
A1 = math.exp(4.768 - 0.8359*math.log(Rs+300)) # Eq 3.125a
A2 = 0.555 + 133.5/(Rs+300)
uob = A1 * uod**A2
print('Live oil viscoity:',round(uob,4), 'cP' )

Live oil viscoity: 0.5458 cP
